In [50]:
import torch
# 이미지데이터를 사용하기 위해서 기존 데이터 사용, 변형
import torchvision.datasets as dataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import random
import torch.optim as optim
import torch.nn.functional as F

In [51]:
learning_rate=0.001
epochs=30
batch_size=256


In [52]:
mnist_train=dataset.MNIST(root="data/MNIST_data", #데이터 저장위치
                          train=True, #train data로 지정
                          transform=transforms.ToTensor(), #array -> toTensor로 변환
                          download=True
                          )
mnist_test=dataset.MNIST(root="data/MNIST_data", #데이터 저장위치
                          train=False, #train data로 지정
                          transform=transforms.ToTensor(), #array -> toTensor로 변환
                          download=True
                          )

In [53]:
print(mnist_train)
print(mnist_test)

Dataset MNIST
    Number of datapoints: 60000
    Root location: data/MNIST_data
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset MNIST
    Number of datapoints: 10000
    Root location: data/MNIST_data
    Split: Test
    StandardTransform
Transform: ToTensor()


In [54]:
data_loader=DataLoader(dataset=mnist_train,
                       batch_size=batch_size,
                       shuffle=True,
                       drop_last=True #뽑은거는 중복사용안함
                       )
test_loader=DataLoader(dataset=mnist_test,batch_size=batch_size,shuffle=True,drop_last=True)


In [55]:
cnt_batch_train=len(data_loader) #234
cnt_batch_test=len(test_loader) #39
print(cnt_batch_train,cnt_batch_test)


234 39


In [56]:
# 한번에 가져오기
for X, Y in data_loader:
  print(X.size(),Y.size())
  break

torch.Size([256, 1, 28, 28]) torch.Size([256])


In [57]:
# 모델 클래스 정의
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    # 첫번째 레이어 합성곱연산
    self.layer1=nn.Sequential(
    nn.Conv2d(1,32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2))
    # 위의 작업 이후에 [32,14,14] 변환됨
    self.layer2=nn.Sequential(
      nn.Conv2d(32,64,kernel_size=3, padding=1, stride=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)
    )
    # 위의 작업이 끝나면 [64,7,7]로 변환됨
    self.fc=nn.Linear(7*7*64, 10, bias=True)
    nn.init.xavier_uniform_(self.fc.weight) #완전연결층 한정 가중치 초기화

  def forward(self,x):
    out=self.layer1(x)
    out=self.layer2(out)
    out=out.view(out.size(0),-1)
    out=self.fc(out)
    return out
  

In [58]:
device='cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [59]:
model=CNN().to(device)
crit=nn.CrossEntropyLoss().to(device)
optimizer=optim.Adam(model.parameters(), lr=learning_rate)

In [60]:
for epoch in range(epochs):
  avg_cost=0

  for X,Y in data_loader:
    X=X.to(device) 
    Y=Y.to(device) 

    y_hat=model(X)
    optimizer.zero_grad()
    cost=crit(y_hat,Y)
    cost.backward()
    optimizer.step()

    avg_cost+=cost/cnt_batch_train
  print('epoch:{}, cost:{}'.format(epoch,avg_cost.item()))  
  


epoch:0, cost:0.3640304505825043
epoch:1, cost:0.08700936287641525
epoch:2, cost:0.0591961145401001
epoch:3, cost:0.04933690279722214
epoch:4, cost:0.04188058152794838
epoch:5, cost:0.03708373382687569
epoch:6, cost:0.030959146097302437
epoch:7, cost:0.028140027076005936
epoch:8, cost:0.024494893848896027
epoch:9, cost:0.023070797324180603
epoch:10, cost:0.020784756168723106
epoch:11, cost:0.01809876039624214
epoch:12, cost:0.01578572578728199
epoch:13, cost:0.013490571640431881
epoch:14, cost:0.012442294508218765
epoch:15, cost:0.012364866212010384
epoch:16, cost:0.009047540836036205
epoch:17, cost:0.007815792225301266
epoch:18, cost:0.008343075402081013
epoch:19, cost:0.008001962676644325
epoch:20, cost:0.005982185713946819
epoch:21, cost:0.006338717881590128
epoch:22, cost:0.006913800723850727
epoch:23, cost:0.0039850566536188126
epoch:24, cost:0.004027584567666054
epoch:25, cost:0.005263627041131258
epoch:26, cost:0.0039057235699146986
epoch:27, cost:0.0033660714980214834
epoch:28,

In [ ]:
with torch.no_grad():
  accuracys=0

  for X, Y in test_loader:
    X=X.to(device)
    Y=Y.to(device)

    pred=model(X)
    accuracy=(pred.argmax(dim=1)==Y).sum()
    accuracys+=accuracy
    print(pred)
    print(Y)
    print(accuracy)
  print(accuracy/cnt_batch_test*batch_size)
